In [74]:
# Evaluation Metrics
from sympy import count_ops
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

def relative_error(actual, predicted):
  mse = mean_squared_error(actual, predicted)
  sqrt = np.sqrt(mse)
  return sqrt/np.mean(actual) 


def complexity(equation):
  l = count_ops(equation)
  return l


def model_score(row):
  return (row[2] + row[5])*50 - row[8]

In [75]:
# Store Equations in Array

# Store:
# Equation, train metrics, test metrics, complexity, model score

learned_equations = []

In [76]:
# Load Data
import pandas as pd
import numpy as np

observed_data = np.array(pd.read_csv('/content/drive/MyDrive/Project/observed.csv'))

X_observed = observed_data[:, [0, 1, 2, 3]]
Y_observed = observed_data[:, 4]

In [77]:
# Split Data (state = 1 for consistency)
from sklearn.model_selection import train_test_split

X_observed_train, X_observed_test, Y_observed_train, Y_observed_test = train_test_split(X_observed, Y_observed, random_state=1, train_size=0.8)

Linear Regression

In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
import sympy

for i in range(len(X_observed[0])):
  new_row = ['Linear Regression: Feature ' + str(i + 1)]
  x_train, x_test = X_observed_train[:, [i]], X_observed_test[:, [i]]
  linreg = LinearRegression()
  linreg.fit(x_train, Y_observed_train)

  eq = sympy.simplify(str(linreg.coef_[0]) + '*x' + str(i + 1) + '+' + str(linreg.intercept_))
  new_row.append(str(eq))

  trainpreds = linreg.predict(x_train)
  new_row.append(r2_score(Y_observed_train, trainpreds))
  new_row.append(mean_absolute_percentage_error(Y_observed_train, trainpreds))
  new_row.append(relative_error(Y_observed_train, trainpreds))

  testpreds = linreg.predict(x_test)
  new_row.append(r2_score(Y_observed_test, testpreds))
  new_row.append(mean_absolute_percentage_error(Y_observed_test, testpreds))
  new_row.append(relative_error(Y_observed_test, testpreds))
  
  new_row.append(complexity(eq))

  new_row.append(model_score(new_row))
  learned_equations.append(new_row)

In [73]:
np.savetxt("drive/MyDrive/Equation Learning/learned_equations-linear-reg.csv", np.array(learned_equations), delimiter=",", fmt='%s')

Add New Features for LASSO and EQL-NN

In [78]:
# Flow Velocity
flow_velocity_observed = np.transpose([np.power(X_observed[:, 2], 4)/(9.806*np.power(X_observed[:, 0], 2))])

# Water Depth
water_depth_observed = np.transpose([np.power(X_observed[:, 1], 3)/np.power(X_observed[:, 3], 4)])

# Fr
fr_observed = flow_velocity_observed/(np.sqrt(9.806*water_depth_observed))

# Re
re_observed = np.multiply(flow_velocity_observed, water_depth_observed)/0.000001

# Re*
re_shear_observed = np.multiply(np.transpose([X_observed[:, 1]]), water_depth_observed)/0.000001

X_observed = np.concatenate((X_observed, flow_velocity_observed, water_depth_observed, fr_observed, re_observed, re_shear_observed), axis=1)

In [82]:
# Check Shape
print(X_observed.shape, Y_observed.shape)

(588, 9) (588,)


In [80]:
# Split Data (state = 1 for consistency)
from sklearn.model_selection import train_test_split

X_observed_train, X_observed_test, Y_observed_train, Y_observed_test = train_test_split(X_observed, Y_observed, random_state=1, train_size=0.8)

Lasso

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

alphas = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
best_score = 0
best_row = []
best_alpha = 0

for alpha in alphas:
  new_row = ['Features Used: LASSO']

  lasso = Lasso(alpha=alpha)
  lasso.fit(X_observed_train, Y_observed_train)

  equation = ''
  i = 0
  for coef in lasso.coef_:
    equation += str(coef) + '*x_' + str(i + 1) + '+'
    i += 1
  equation += str(lasso.intercept_)
  eq = sympy.simplify(equation)

  new_row.append(str(eq))

  trainpreds = lasso.predict(X_observed_train)
  new_row.append( r2_score(Y_observed_train, trainpreds))
  new_row.append(mean_absolute_percentage_error(Y_observed_train, trainpreds))
  new_row.append(relative_error(Y_observed_train, trainpreds))

  testpreds = lasso.predict(X_observed_test)
  new_row.append(r2_score(Y_observed_test, testpreds))
  new_row.append(mean_absolute_percentage_error(Y_observed_test, testpreds))
  new_row.append(relative_error(Y_observed_test, testpreds))
  
  new_row.append(complexity(eq))

  if model_score(new_row) > best_score:
    new_row.append(model_score(new_row))
    best_row = new_row
    best_alpha = alpha
learned_equations.append(best_row)
print('Best Alpha:', best_alpha)

In [62]:
np.savetxt("drive/MyDrive/Equation Learning/learned_equations-LASSO.csv", np.array(learned_equations), delimiter=",", fmt='%s')

EQL-NN

In [ ]:
# Load EQL-NN
!pip install EQL-NN
from EQL.model import EQL
import tensorflow as tf

In [ ]:
# Convert EQL-NN Weights + Biases to Equation
import sympy
funcs = ['id', 'sin', 'cos', 'sig', 'mult']
def generate_learned_equation(EQLmodel, hidden_layers, num_features, funcs, exclude):
  # Initialize y
  y = []
  for f in range(num_features):
    y.append('x_' + str(f + 1))

  for l in range(hidden_layers + 1):
    layer = EQLmodel.get_weights(layer=l + 1)

    # Multiply weights
    weights = np.transpose(layer[0])
    z = []
    for row in weights:
      new_row = ''
      for i in range(len(row)):
        if i == 0:
          new_row += str(row[i]) + '*(' + y[i] + ')'
        else:
          new_row += ' + ' + str(row[i]) + '*(' + y[i] + ')'
      z.append(new_row)

    # Add biases
    biases = layer[1]
    for i in range(len(biases)):
      z[i] = str(z[i]) + ' + ' + str(biases[i])

    # Apply functions (Skip if last layer)
    if l != hidden_layers:

      # Get functions for this layer
      if len(exclude) > 0:
        exclude_funcs = exclude[l]
        layer_funcs = [func for func in funcs if func not in exclude_funcs]
      else:
        layer_funcs = funcs

      new_y = []
      f_functions = z[:-2]
      g_functions = z[-2:]

      for i in range(len(layer_funcs)):
        if layer_funcs[i] == 'id':
          new_y.append(z[i])
        elif layer_funcs[i] != 'mult':
          new_y.append(layer_funcs[i] + '(' + z[i] + ')')
        elif layer_funcs[i] == 'mult':
          new_y.append('(' + z[i] + ')' + '*(' + z[i+1] + ')')
      y = new_y

    else:
      y = z

  return sympy.simplify(y[0])

In [ ]:
# Recursively Get All Combos of features
features = [1, 2, 3, 4, 5, 6, 7, 8, 9]

def combinations(l):
    if l:
      result = combinations(l[:-1])
      return result + [c + [l[-1]] for c in result]
    else:
      return [[]]

combos = combinations(features)

In [ ]:
print(len(combos))

512


In [ ]:
# Loop through each variable + trig or no trig (have to do division separately)
tracker = 0
for combo in combos:
  print(tracker)
  if len(combo) > 0:
    label = 'Feature(s):'
    for feature in combo:
      label += ' ' + str(feature)
    print(label)
    # No Trig
    new_row = [label]

    x_train = X_observed_train[:, np.array(combo) - 1]
    x_test = X_observed_test[:, np.array(combo) - 1]

    EQLmodel = EQL(num_layers=1, dim=len(combo))
    EQLmodel.build_and_compile_model(exclude=[['sin', 'cos', 'sig']])
    EQLmodel.fit(x_train, Y_observed_train, t0=2000, t1=1800, t2=600, lmbda=0.001, batch_size=32)

    equation = generate_learned_equation(EQLmodel, 1, len(combo), funcs, [['sin', 'cos', 'sig']])
    new_row.append(str(equation))

    trainpreds = EQLmodel.predict(x_train)
    try:
      new_row.append(r2_score(Y_observed_train, trainpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(mean_absolute_percentage_error(Y_observed_train, trainpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(relative_error(Y_observed_train, trainpreds))
    except:
      new_row.append('-')

    testpreds = EQLmodel.predict(x_test)
    try:
      new_row.append(r2_score(Y_observed_test, testpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(mean_absolute_percentage_error(Y_observed_test, testpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(relative_error(Y_observed_test, testpreds))
    except:
      new_row.append('-')

    new_row.append(complexity(equation))

    learned_equations.append(new_row)

    # Trig
    new_row = [label]

    x_train = X_observed_train[:, np.array(combo) - 1]
    x_test = X_observed_test[:, np.array(combo) - 1]


    EQLmodel = EQL(num_layers=1, dim=len(combo))
    EQLmodel.build_and_compile_model(exclude=['sig'])
    EQLmodel.fit(x_train, Y_observed_train, t0=2000, t1=1800, t2=600, lmbda=0.001, batch_size=32)

    equation = generate_learned_equation(EQLmodel, 1, len(combo), funcs, [['sig']])
    new_row.append(str(equation))

    trainpreds = EQLmodel.predict(x_train)
    try:
      new_row.append(r2_score(Y_observed_train, trainpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(mean_absolute_percentage_error(Y_observed_train, trainpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(relative_error(Y_observed_train, trainpreds))
    except:
      new_row.append('-')

    testpreds = EQLmodel.predict(x_test)
    try:
      new_row.append(r2_score(Y_observed_test, testpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(mean_absolute_percentage_error(Y_observed_test, testpreds))
    except:
      new_row.append('-')
    try:
      new_row.append(relative_error(Y_observed_test, testpreds))
    except:
      new_row.append('-')

    new_row.append(complexity(equation))

    new_row.append(model_score(new_row))

    learned_equations.append(new_row)

    np.savetxt("drive/MyDrive/Equation Learning/learned_equations.csv", np.array(learned_equations), delimiter=",", fmt='%s')

  tracker += 1